In [1]:
import os
import re
from glob import glob
from tqdm import tqdm
import javalang
import pandas as pd
import time
from requests.exceptions import SSLError
def rate_limited():
        print("Connection Error: maybe rate limited, waiting 1 minute")
        time.sleep(60)

def get_source_code(commitId, project):
    import random
    import requests
    from robobrowser import RoboBrowser

    HEADERS_LIST = [
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; x64; fr; rv:1.9.2.13) Gecko/20101203 Firebird/3.6.13',
        'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
        'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
        'Mozilla/5.0 (Windows NT 5.2; RW; rv:7.0a1) Gecko/20091211 SeaMonkey/9.23a1pre'
    ]

    link = []
    codes = []

    retries = 0
    
    
    while(len(codes)<1):
     #   if retries > 1:
          #  print("Retrying", retries, "/ 4")
        if retries > 3:
            print(commitId, project)
            break
        session = requests.Session()
        browser = RoboBrowser(session=session, user_agent=random.choice(HEADERS_LIST), parser="lxml")
        url = "https://github.com/" + project.replace("-", "/") + "/commit/" + commitId

        try:
            browser.open(url + "?diff=unified")
        except SSLError:
            rate_limited()
            pass
        results = browser.find_all("a")
        for item in results:
            if item.string is not None:
                if ".java" in item.string:
                    try:
                     #   print(item.string, project, commitId)
                        second_url = "https://raw.githubusercontent.com/" + project.replace("-", "/") + "/" + commitId + "/" + item.string
                        browser.open(second_url)
                        codes.append(browser.find().text)
                    except (SSLError):
                        rate_limited()
                        pass

        retries = retries + 1

    return codes
    



def removeComments(string):
    string = re.sub(re.compile("/\*.*?\*/", re.DOTALL), "",
                    string)  # remove all occurance streamed comments (/*COMMENT */) from string
    string = re.sub(re.compile("//.*?\n"), "",
                    string)  # remove all occurance singleline comments (//COMMENT\n ) from string
    return string


temp_subfiles = []

pattern = "*.java"

for dir, _, _ in os.walk("/home/manny/PycharmProjects/GithubScraper/java_code_files"):
    temp_subfiles.extend(glob(os.path.join(dir, pattern)))


df = pd.read_csv("1151-commits-labeled-with-maintenance-activities.csv", sep="#")

code = {"JavaSyntaxError":[],"Source_code":[],"code_col" :[],"code_line" :[],"code" : [],"astcode": [], "label": [], "repoSource": [], 'add': [], 'allow': [], 'bug': [], 'chang': [], 'error': [],
        'fail': [], 'fix': [], 'implement': [],
        'improv': [], 'issu': [], 'method': [], 'new': [], 'npe': [], 'refactor': [], 'remov': [], 'report': [],
        'set': [], 'support': [], 'test': [], 'use': [],'database_source':[]}

print(df.columns)

def dict_append(code, key, value):
    if value == None:
        code[key].append("nan")
    else:
        code[key].append("value")
        return 

for index, row in tqdm(df.iterrows()):
    source_codes = get_source_code(row['commitId'], row['project'])
    for source_code in source_codes:
        try:
            tree = javalang.parse.parse(source_code)
            for codeblock in tree.children:
                try:
                    code['code'].append(codeblock._position._source)
                except AttributeError:
                    code['code'].append(None)
                try:
                    code['code_line'].append(codeblock._position.line)
                except AttributeError:
                    code['code_line'].append(None)
                try:
                    code['code_col'].append(codeblock._position.column)
                except AttributeError:
                    code['code_col'].append(None)

                code['astcode'].append(codeblock)
                code['Source_code'].append(source_code)
                code['label'].append(row['label'])
                code['repoSource'].append(row['project'])
                code['add'].append(row['add'])
                code['allow'].append(row['allow'])
                code['bug'].append(row['bug'])
                code['chang'].append(row['chang'])
                code['error'].append(row['error'])
                code['fail'].append(row['fail'])
                code['fix'].append(row['fix'])
                code['implement'].append(row['implement'])
                code['improv'].append(row['improv'])
                code['issu'].append(row['issu'])
                code['method'].append(row['method'])
                code['new'].append(row['new'])
                code['npe'].append(row['npe'])
                code['refactor'].append(row['refactor'])
                code['remov'].append(row['remov'])
                code['report'].append(row['report'])
                code['set'].append(row['set'])
                code['support'].append(row['support'])
                code['test'].append(row['test'])
                code['use'].append(row['use'])
                code['database_source'].append("Promise")
                code['JavaSyntaxError'].append(False)
        except (javalang.parser.JavaSyntaxError, KeyError):
                code['code'].append(source_code)
                code['astcode'].append(None)
                code['Source_code'].append(source_code)
                code['code_line'].append(None)
                code['code_col'].append(None)
                code['JavaSyntaxError'].append(True)
                code['label'].append(row['label'])
                code['repoSource'].append(row['project'])
                code['add'].append(row['add'])
                code['allow'].append(row['allow'])
                code['bug'].append(row['bug'])
                code['chang'].append(row['chang'])
                code['error'].append(row['error'])
                code['fail'].append(row['fail'])
                code['fix'].append(row['fix'])
                code['implement'].append(row['implement'])
                code['improv'].append(row['improv'])
                code['issu'].append(row['issu'])
                code['method'].append(row['method'])
                code['new'].append(row['new'])
                code['npe'].append(row['npe'])
                code['refactor'].append(row['refactor'])
                code['remov'].append(row['remov'])
                code['report'].append(row['report'])
                code['set'].append(row['set'])
                code['support'].append(row['support'])
                code['test'].append(row['test'])
                code['use'].append(row['use'])
                code['database_source'].append("Promise")
                pass


/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/manny/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
0it [00:00, ?it/s]

Index(['commitId', 'project', 'comment', 'label',
       'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_CLASS_DERIVABILITY',
       'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS',
       'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE',
       'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT',
       'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING',
       'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE',
       'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE',
       'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE',
       'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING',
       'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE',
       'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE',
       'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE',
       'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS',
       'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE',

101it [02:14,  1.33s/it]

013fd99d0dd0df30e0bb7fcb98e41cd1633319cb hbase


102it [02:15,  1.32s/it]

...

381ccde48d11ef82c371789898e563233141b7ee spring-framework


1150it [16:22,  1.17it/s]

40822746306f5d0e5d553504275046ec4906309b spring-framework


1151it [16:23,  1.17it/s]

e5b505224b77bb2428ab9ca85e894a1ba51f8994 spring-framework


In [2]:
print(code)
for key,value code.items():
    print(key, str(len(code[key])))

SyntaxError: invalid syntax (<ipython-input-2-89d8d5feb440>, line 2)

In [10]:
df = pd.DataFrame.from_dict(code)

In [ ]:
print(code)

In [11]:
df.to_csv("promise_gzip.gz", index=True, compression='gzip' )

In [12]:
pd.DataFrame.from_csv("promise_gzip.gz").columns.values

/home/manny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


array(['JavaSyntaxError', 'Source_code', 'code_col', 'code_line', 'code',
       'astcode', 'label', 'repoSource', 'add', 'allow', 'bug', 'chang',
       'error', 'fail', 'fix', 'implement', 'improv', 'issu', 'method',
       'new', 'npe', 'refactor', 'remov', 'report', 'set', 'support',
       'test', 'use', 'database_source'], dtype=object)